In [ ]:
import pathlib
import sys
import numpy as np
import pandas as pd
import rasterio
import refet
import pyproj
import xarray
import requests
import multiprocessing as mp
import configparser
import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

In [ ]:
#sys.path.append("//")
sys.path.append("../../micromet")
import micromet
#from micromet.volk import ffp_climatology as ffp
import micromet.volk as ffp
from micromet import AmerifluxDataProcessor

In [ ]:
import logging

In [ ]:
config_path = pathlib.Path(f'../../station_config')

# iterate through all of the station configuration files
for stationfile in config_path.glob("US*.ini"):

    station = stationfile.stem
    print(station)
    outfolder = station.replace('-','').lower()

    # Specify the directory path
    out_dir = pathlib.Path(f"./output/{outfolder}")
    print(out_dir.exists())
    # Check if the directory exists; if not, create it (including parent directories)
    if not out_dir.exists():
        out_dir.mkdir(parents=True)
    else:
        pass

    # calculate the hourly footprint using Kljun
    micromet.calc_hourly_ffp(station,
                         startdate='2021-01-01',
                         config_path=config_path,
                         out_dir=out_dir,
                         secrets_path='../../secrets/config.ini',
                         epsg=5070,
                         h_c=0.2,
                         zm_s=2.0,
                         dx=5.0,
                         h_s=2000.0,
                         origin_d=500.0,
                             )

    # generate weighted daily rasters
    micromet.weighted_rasters(stationid=station,
                              start_hr=6,
                              normed_NLDAS_stats_path='output/nldas_all_normed.parquet',
                              out_dir=out_dir,
                              )


In [ ]:
config_path = pathlib.Path(f'../../station_config')
# iterate through all of the station configuration files
statpolys = {}

for stationfile in config_path.glob("US*.ini"):

    station = stationfile.stem
    print(station)
    outfolder = station.replace('-','').lower()

    # Specify the directory path
    out_dir = pathlib.Path(f"./output/{outfolder}")
    gdfs = {}
    for rasterfile in out_dir.glob("*weighted.tif"):
       rdate = rasterfile.stem.split("_")[0]
       gdfs[rdate] = micromet.outline_valid_cells(rasterfile, out_file=None)

    statpolys[station] = pd.concat(gdfs)
    statpolys[station].to_file(f"{station}_weighted_raster.geojson")


In [ ]:
station = "US-UTM"
model = 'ensemble'
pathnm = station.lower().replace('-','')

dir1 = pathlib.Path(f"./output/{pathnm}/")    # e.g., contains '...20210305.tif', etc.

dir2 = pathlib.Path("G:/My Drive/OpenET Exports/")

out_dir = pathlib.Path(f"./output/{pathnm}_mult/")

tsum = micromet.multiply_directories_rast(dir1, dir2, out_dir=out_dir,model=model)

series= pd.Series(tsum)

metadata = micromet.load_configs(station,
                 config_path='../../station_config/',
                 secrets_path="../../secrets/config.ini")
df = micromet.fetch_and_preprocess_data(metadata["url"], station, startdate='2023-01-01')
s = df['et']

# replace values less than 0 with 0
s = s.where(s >= 0, 0)
# resample with daily, only keeping days with more than 19 records.
daily_stat_et = s.dropna().resample('D').sum(min_count=20)

In [ ]:
et_compare = pd.concat([daily_stat_et,series],axis=1)
et_compare.columns = ['station_mm','ens_mm']
plt.scatter(et_compare['station_mm'],et_compare['ens_mm'])
plt.xlabel('Raw Daily Station ET (mm)')
plt.ylabel(f'{model} ET (mm)')
plt.title(f"{station} from {et_compare.first_valid_index():%Y-%m-%d} to {et_compare.last_valid_index():%Y-%m-%d}")
plt.savefig(f"{station}_ET.pdf")
plt.savefig(f"{station}_ET.png")